In [1]:
import numpy as np
import re
import itertools
from collections import Counter
from tensorflow.contrib import learn
import unittests as tests
import tensorflow as tf

# Funcoes Auxiliares (tiradas do paper)

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

# Data explore

In [3]:
import random

data,labels = load_data_and_labels('data/rt-polaritydata/rt-polarity.pos','data/rt-polaritydata/rt-polarity.neg')

print (len(data)) #todos os reviews, cada elemento eh um review
print (len(labels)) #todos os sentimentos


10662
10662


In [4]:
(data[0], labels[0]) #tupla primeiro review, no caso eh positivo

("the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal",
 array([0, 1]))

# Pre processing
- Build vocabulary

In [5]:
max_review_size = max([len(x.split(" ")) for x in data])

#Maps documents to sequences of word ids.
#Basicamente cada palavra ja eh mapeada para um id e a sentenca tb
vocab_processor = learn.preprocessing.VocabularyProcessor(max_review_size)
x = np.array(list(vocab_processor.fit_transform(data)))#aqui que ocorre o mapeamento
(x[0], data[0])

(array([ 1,  2,  3,  4,  5,  6,  1,  7,  8,  9, 10, 11, 12, 13, 14,  9, 15,
         5, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]),
 "the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal")

In [6]:
max_review_size

56

- Randomly shuffle data

In [7]:
#Shuffle Data
np.random.seed(10) #for debugging, garante que os numeros aleatorios gerados sempre sejam os mesmos
shuffle_indices = np.random.permutation(np.arange(len(labels)))
shuffle_indices

array([ 7359,  5573, 10180, ...,  1344,  7293,  1289])

In [8]:
# basicamente eu refaço o array de arrays, mas agora cada indice eh oq ta como indice gerado do random
#ex: x[7359] == x_shuffled[0] assim os dois (data, label) mantem o mesmo indice
x_shuffled = x[shuffle_indices] #aqui sao as sentencas ja mapeadas
y_shuffled = labels[shuffle_indices]



 - Train/Validation split

In [9]:
val_percentage = .1
val_sample_index = -1 * int(val_percentage * float(len(labels)))
#TODO - fazer do meu jeito isso aqui
#justamente pra pegar os ultimos enquanto no validation, pega de ordem reversa,
#ex: pegue todos com excessao dos -1066 ultimos
#ou pegue os 1066 primeiros

In [10]:
x_train, x_val = x_shuffled[:val_sample_index], x_shuffled[val_sample_index:]
y_train, y_dev = y_shuffled[:val_sample_index], y_shuffled[val_sample_index:]

In [11]:
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 18758
Train/Dev split: 9596/1066


# Build CNN

- Inputs and Labels instances

In [12]:
def neural_net_sentence_input(sentence_size):
    """
    Return a Tensor for a batch of image input
    : sentence_size: Size of the sentence with the biggest len
    : return: Tensor for sentences input.
    Remeber: all sentences were padded to get the max len
    """
    # TODO: Implement Function
    return tf.placeholder(tf.int32, shape=[None,sentence_size],name='input_x')


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=[None,n_classes],name='input_y')


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=None,name='keep_prob')


"""
UNIT TESTS
"""
tf.reset_default_graph()
tests.test_nn_sentence_inputs(neural_net_sentence_input)
tests.test_nn_label_inputs(neural_net_label_input)
tests.test_nn_keep_prob_inputs(neural_net_keep_prob_input)

Sentence Input Tests Passed.
Label Input Tests Passed.
Keep Prob Tests Passed.


- Embedding Layer

In [13]:
def embedding_creation(x_tensor,vocab_size,embedding_size):
    W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),name="W")
    embedded_chars = tf.nn.embedding_lookup(W, x_tensor)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    
    return embedded_chars_expanded


tests.test_embed(embedding_creation)

Tests Passed


- Convolution Layer

In [14]:
def conv2d_maxpool(x_tensor, num_filters, filter_size):
    """
    return: A tensor that represents convolution and max pooling of x_tensor
    """
    embbeding_size = int(x_tensor.shape[2])
    filter_shape = [filter_size,embbeding_size, 1, num_filters]
    
    weights = tf.Variable(tf.truncated_normal(filter_shape,stddev=0.1), name="W")
    bias = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    """
    Strides controls how the filter convolves around the input
    As we want to go each word per time, everything will have size one
    As we apply conv layers, we could pad the image to preserve the dimension 
    (and try to extract more level features)
    Because we are only dealing with words, this would not be necessary. This is known as narrow convolution
    
    Conv gives us an output of shape [1, sequence_length - filter_size + 1, 1, 1] - There is a formula to discover that
    
    """
    conv = tf.nn.conv2d(x_tensor, weights, strides=[1, 1, 1, 1], padding='VALID')

    conv = tf.nn.bias_add(conv, bias)
    #add non linearity
    h = tf.nn.relu(conv, name="relu")
    sequence_length = int(x_tensor.shape[1])
    conv_output = [1, sequence_length - filter_size + 1, 1, 1]
    
    #Maxpooling over the outputs
    #this will heaturn a tensor of shape [batch_size, 1, 1, num_filters] 
    #which is essencialy a feature vector where the last dimension correspond to features
    #Stride have this size basically because of the same logic applied before
    pooled = tf.nn.max_pool(h, ksize=conv_output,
                            strides=[1, 1, 1, 1],
                            padding='VALID',
                            name='pool') 
    
    return pooled


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_con_pool(conv2d_maxpool)

Tests Passed


- Apply different filters

In [15]:
def apply_conv_filters(x_tensor,filter_sizes,num_filters):
# Create a convolution + maxpool layer for each filter size
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            pooled = conv2d_maxpool(x_tensor, num_filters, filter_size)
            pooled_outputs.append(pooled)     
    num_filters_total = num_filters * len(filter_sizes)
    #concat -> sum(Daxis(i)) where Daxis is Dimension axis (in our case is the third one)
    h_pool = tf.concat(pooled_outputs, 3)
    return h_pool

tests.test_apply_filters(apply_conv_filters,conv2d_maxpool)

Tests Passed


- Flatten Layer

The flatten function to change the dimension of x_tensor from a 4-D tensor to a 2-D tensor. The output should be the shape (Batch Size, Flattened Features Size).

In [16]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    # TODO: Implement Function
    #print (tf.contrib.layers.flatten(x_tensor))
    #This is a general flatten function
    flat = x_tensor.shape[1]*x_tensor.shape[2]*x_tensor.shape[3]
    return tf.reshape(x_tensor,[-1,int(flat)])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_flatten(flatten)

Tests Passed


In [17]:
def calculate_output(x_tensor,num_classes):
    num_filters_total = int(x_tensor.shape[1])
    W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
#    W = tf.get_variable("W", shape=[num_filters_total, num_classes],initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
#     l2_loss += tf.nn.l2_loss(W)
#     l2_loss += tf.nn.l2_loss(b)
    scores = tf.nn.xw_plus_b(x_tensor, W, b, name="scores")
    predictions = tf.argmax(scores, 1, name="predictions")
            
    return (scores, predictions)
tests.test_compute_output(calculate_output)

Tests Passed


- Convolutional Network

In [ ]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    conv_num_filters = 32
    conv_ksize = (5,5) #filter size
    conv_strides = (1,1) #strides
    pool_ksize = (2, 2)
    pool_strides = (2, 2)
    conv_layer = conv2d_maxpool(x, conv_num_filters, conv_ksize, conv_strides, pool_ksize, pool_strides)
    
    conv_num_filters = 64 
    conv_ksize = (5,5) #filter size
    conv_strides = (1,1) #strides
    pool_ksize = (2, 2)
    pool_strides = (4, 4)
    conv_layer = conv2d_maxpool(x, conv_num_filters, conv_ksize, conv_strides, pool_ksize, pool_strides)
    

    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    flat_layer = flatten(conv_layer)
    flat_layer =  tf.nn.dropout(flat_layer, keep_prob)
    

    # TODO: Apply 1, 2, or 3 Fully Connected Layers
    #    Play around with different number of outputs
    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    
    num_outputs = 100
    fully_conn_layer = fully_conn(flat_layer, num_outputs)
    fully_conn_layer = tf.nn.dropout(fully_conn_layer, keep_prob)
    
    
    # TODO: Apply an Output Layer
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    
    num_classes = 10
    output_layer = output(fully_conn_layer, num_classes)

    
    # TODO: return output
    return output_layer

Build The Neural Network